# EDA Datset Grammys

- En este jupyter notebook veras todo el proceso de extraccion, limpieza y carga de uno de los 2 datasets del proyecto, en este caso el dataset de los grammys el cual se extrae desde una base de datos para poder procesarlo en este notebook y al final de toda su transformacion guardarlo en un archivo pickle que nos permitira interactuar con el desde otros notebooks.
- La desicion de usar la biblioteca pickle se tomo por su facilidad de uso y trabajo en este entorno de jupyter y lenguaje de programacion python

## Importacion de bibliotecas
### Usadas para la conexion y manejo de la base de datos
- Sqlaclhemy y create engine: Estas bibliotecas nos ayudan a poder interactuar con nuesta base de datos mysql desde archivos de lenguaje y nos ayudan con el proceso de crear conexiones y consultas a las tablas de nuestras bases de datos.

- Dotenv, load_dotenv y os: Bibliotecas que nos ayuda al manejo de credenciales en archivos publicos como este notebook, como estamos usando variables de entorno para la conexion con mysql, estas bibliotecas son las que nos ayudan a poder usar nuestras credenciales llamandolas desde un archivo ".env".


### Usadas para el analisis exploratiorio y estadistico del dataset
- Pandas y matplotlib: Bibliotecas de alto rendimiento para el manejo y analisis exploratorio de datos permitiendonos una limpieza produnda y organizada con nuestros datasets, ademas de todo el campo de analisis estadistico y grafico profundo que se complementa con matplotlib.

- Pickle: Biblioteca altamente utlizada para la trasnferencia de datos y procesos dentro del lenguaje python ofreciendonos una flexibilidad y facil uso de la persistencia de datos.

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pickle
import pymysql

## Carga de las variables de entorno ( Variables que nos permitiran poder conectarnos con nuestra base de datos

In [2]:
load_dotenv()


True

In [3]:

db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_database = os.getenv("DB_DATABASE")

## Connection Established with PostgreSQL using SQLAlchemy
- Como parte del codigo buscamos a final que nos devuelva el encabezado y la informacion de la tabla solicitada para poder comprobar que el proceso fue exitoso

In [4]:
try:
    Workshop_2_mysql_connection_str = f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_database}'
    Workshop_2_mysql_db_connection = create_engine(Workshop_2_mysql_connection_str)
    print("Conexión establecida con la base de datos...")

    table_name = 'grammy_awards'

    query = f'SELECT * FROM {table_name}'

    grammy_awards_df = pd.read_sql(query, con=Workshop_2_mysql_db_connection)

    print(grammy_awards_df.head())
    print(grammy_awards_df.info())

except Exception as e:
    print(f"Fallo al conectar a la base de datos MySQL: {e}")

finally:
    if 'Workshop_2_mysql_db_connection' in locals():
        Workshop_2_mysql_db_connection.dispose()
    print("Conexión de datos y carga realizada correctamente, conexión cerrada.")



Conexión establecida con la base de datos...
   year                              title               published_at  \
0  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   
1  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   
2  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   
3  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   
4  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   

                  updated_at            category     nominee         artist  \
0  2020-05-19T05:10:28-07:00  Record Of The Year     Bad Guy  Billie Eilish   
1  2020-05-19T05:10:28-07:00  Record Of The Year     Hey, Ma       Bon Iver   
2  2020-05-19T05:10:28-07:00  Record Of The Year     7 rings  Ariana Grande   
3  2020-05-19T05:10:28-07:00  Record Of The Year  Hard Place         H.E.R.   
4  2020-05-19T05:10:28-07:00  Record Of The Year        Talk         Khalid   

                                         

# Busqueda de valores nulos
- De igual manera que en el dataset de grammys podemos ver como algunos campos se detectan como nulos, estos campos son 'nominee', 'artist', 'workers', 'img'

In [ ]:
# Verificar si hay valores nulos en el DataFrame
null_counts = grammy_awards_df.isnull().sum()
print("Recuento de valores nulos por columna:")
print(null_counts)


## Eliminacion de  registros con valores nulos en los campos ('nominee', 'artist', 'workers', 'img')

In [ ]:
# Eliminar filas con valores nulos en las columnas especificadas del DataFrame original
grammy_awards_df.dropna(subset=['nominee', 'artist', 'workers', 'img'], inplace=True)

# Verificar si se eliminaron correctamente los valores nulos en el DataFrame original
print(grammy_awards_df.isnull().sum())
print(grammy_awards_df.info())


# Campos 'published_at' y 'update_at'
- Estos campos tienen registros que corresponden a fechas pero se estan trabajando coomo datos de tipo 'objeto'
- Analicemos un poco mas de como se estructuran los registros de estos campos

In [ ]:
print("Registros en el campo 'published_at':")
print(grammy_awards_df['published_at'])

print("\nRegistros en el campo 'updated_at':")
print(grammy_awards_df['updated_at'])


## Cambio de tipo 'objeto' a 'datatime' para los campos 'published_at' y 'updated_at'

In [ ]:
grammy_awards_df['published_at'] = pd.to_datetime(grammy_awards_df['published_at'], utc=True)
grammy_awards_df['updated_at'] = pd.to_datetime(grammy_awards_df['updated_at'], utc=True)
print(grammy_awards_df.info())


## Alternativa de guardado en csv
- Esta linea la colocamos para la persona que quiera trabar con los dataframes ya limpios como csv, pero recordemos que el proceso del merge se hizo gracias a que los datasets de grammys y de spotify se guardaron como archivos pickle que permitieron el manejo de trabajo de datasets en diferentes notebooks

In [ ]:
#print(grammy_awards_df.info())
# Guardar el DataFrame en un archivo CSV
#grammy_awards_df.to_csv('grammy_dataset_EDA.csv', index=False)

In [ ]:
# Leer el archivo CSV en un DataFrame y parsear las fechas
#grammy_awards_EDA = pd.read_csv('C:/Users/fedeg/OneDrive/Escritorio/Workshop_ETL2/Notebooks/grammy_dataset_EDA.csv', parse_dates=['published_at', 'updated_at'])

# Mostrar información del DataFrame
#print(grammy_awards_EDA.info())

## Guardando dataset limpio como archivo pickle

In [ ]:
# Supongamos que spotify_dataset es tu DataFrame limpio en tu primer notebook
# Guardar el DataFrame como archivo pickle
with open('grammy_awards_df.pkl', 'wb') as f:
    pickle.dump(grammy_awards_df, f)


In [ ]:
# Supongamos que el campo se llama 'album_name'
album_names_unique = grammy_awards_df['artist'].unique()

# Organiza los valores únicos
album_names_unique_sorted = sorted(album_names_unique)

# Imprime los valores únicos ordenados
for album_name in album_names_unique_sorted:
    print(album_name)

In [ ]:
print(grammy_awards_df.info())

In [ ]:
# Obtener los valores únicos de la columna 'winner' y contar cuántas veces aparece cada uno
conteo_winners = grammy_awards_df['winner'].value_counts()

# Imprimir el resultado
print("Valores únicos de 'winner':")
print(conteo_winners)